In [1]:
import os 
import yaml
import tensorflow as tf 
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go 

In [2]:
os.chdir('..')

from utils.VAE_utils import VAE, build_variational_encoder, build_variational_decoder, get_latent_space

### Setup encoder & decoder configs

In [3]:
with open(os.path.join(os.getcwd(), "configs/VAE_config.yaml"), "r") as file:
    vae_config = yaml.safe_load(file)

    
encoder_config = vae_config['encoder']
decoder_config = vae_config['decoder']

### Load data - MNIST 

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [5]:
mnist_digits = np.concatenate([X_train, X_train], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

### Build encoder and decoder

In [6]:
encoder_input, encoder_output, encoder, z_mean, z_log_var = build_variational_encoder(
    encoder_config=encoder_config,
)

In [7]:
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mnist_inputs (InputLayer)       [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
encoder_conv_1 (Conv2D)         (None, 14, 14, 32)   320         mnist_inputs[0][0]               
__________________________________________________________________________________________________
batch_norm_encoder_1 (BatchNorm (None, 14, 14, 32)   128         encoder_conv_1[0][0]             
__________________________________________________________________________________________________
leaky_relu_encoder_1 (LeakyReLU (None, 14, 14, 32)   0           batch_norm_encoder_1[0][0]       
____________________________________________________________________________________________

In [8]:
decoder_input, decoder_output, decoder = build_variational_decoder(
    decoder_config=decoder_config
)

In [9]:
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 2)]               0         
_________________________________________________________________
shape_prod (Dense)           (None, 3136)              9408      
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_1 (Conv2DTransp (None, 14, 14, 64)        36928     
_________________________________________________________________
batch_norm_decoder_1 (BatchN (None, 14, 14, 64)        256       
_________________________________________________________________
leaky_relu_decoder_1 (LeakyR (None, 14, 14, 64)        0         
_________________________________________________________________
decoder_conv_2 (Conv2DTransp (None, 28, 28, 32)        1846

### Instantiate VAE class and train

In [10]:
variational_autoencoder = VAE(encoder, decoder)
variational_autoencoder.compile(
    optimizer=tf.keras.optimizers.Adam()
)

history = variational_autoencoder.fit(
    mnist_digits, 
    epochs=10, 
    batch_size=128
)

Epoch 1/10
938/938 [==============================] - 102s 109ms/step - loss: 226.3839 - reconstruction_loss: 183.5345 - kl_loss: 1.3480
Epoch 2/10
938/938 [==============================] - 108s 115ms/step - loss: 161.3226 - reconstruction_loss: 158.2419 - kl_loss: 1.1706
Epoch 3/10
938/938 [==============================] - 100s 107ms/step - loss: 155.8638 - reconstruction_loss: 153.4755 - kl_loss: 1.4102
Epoch 4/10
938/938 [==============================] - 101s 108ms/step - loss: 152.6191 - reconstruction_loss: 150.1099 - kl_loss: 1.6141
Epoch 5/10
938/938 [==============================] - 102s 109ms/step - loss: 150.2410 - reconstruction_loss: 147.8045 - kl_loss: 1.7660
Epoch 6/10
938/938 [==============================] - 103s 109ms/step - loss: 148.4064 - reconstruction_loss: 146.0237 - kl_loss: 1.8863
Epoch 7/10
938/938 [==============================] - 102s 109ms/step - loss: 147.1320 - reconstruction_loss: 144.7413 - kl_loss: 1.9884
Epoch 8/10
938/938 [=====================

In [11]:
latent_space, sample_range_x, sample_range_y, pixel_range = get_latent_space(
    decoder=decoder, 
    n=30, 
    digit_size=28,
    scale=1.5
)

In [18]:
fig = go.Figure()

fig.add_trace(
    go.Heatmap(
        x=np.arange(len(sample_range_x)),
        y=np.arange(len(sample_range_y)),
        z=latent_space,
        showscale=False
    )
)


fig.update_layout(
    height=800,
    width=800,
    margin=dict(b=0, l=0, r=0, t=20),
    title=dict(
        text='MNIST Digits Represented with a Multivariate Gaussian | Variational Autoencoder',
        font=dict(size=11),
    ),
    xaxis=dict(
        title=dict(text='z[0]'),
        tickmode='array',
        tickfont=dict(size=10),
        tickvals=pixel_range,
        ticktext=sample_range_x
    ),
    yaxis=dict(
        title=dict(text='z[1]'),
        tickmode='array',
        tickfont=dict(size=10),
        tickvals=pixel_range,
        ticktext=sample_range_y,
        autorange='reversed'
    ),
)

fig.write_html('./docs/VAE_mnist.html')

In [20]:
fig.write_image('./VAE_mnist.png')